In [ ]:
%pip install pyserial pandas matplotlib

: 

In [ ]:
#Importaciones y configuración
import serial
import time
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from datetime import datetime
from IPython.display import clear_output

In [ ]:
#Calibración de sensor
# Lista de valores cuando el sensor está en suelo completamente húmedo (wet)
valores_wet = [
    355,193,336,417,299,238,352,404,252,247,348,421,324,196,314,374,
    362,201,363,203,292,362,379,214,282,362,387,220,283,368,368,185,
    327,365,353,183,331,416,283,266,367,340,216,350,388,189,332,419,
    267,276,375,334,209,343,419,250,276,378,352,197,340,414,278,256,
    348,370,228,346,268,283,322,318,242,297,327,314,239,301,334,304,
    251,314,341,248,308,347,263,296,315,328,244,304,342,283,277,324,
    304,259,316,313,251,314,324,239,320,321,240,310,346,282,273,321,
    313,248,313,336,245,307,347,266,337,388,213,334,389,226,307,401,
    303,248,353,360,194,331,412,268,274,368,342,246,341,209,334,403,
    260,269,350,348,202,321,384,293,250,343,349,199,337,388
]

# Cálculo del valor promedio (wet)
promedio_wet = round(sum(valores_wet) / len(valores_wet))
print(f"🔹 Valor promedio 'wet': {promedio_wet}")


In [ ]:
#Definición de funciones auxiliares
def iniciar_arduino(puerto='COM7', baudios=9600):
    """Abre el puerto serie y espera para que Arduino inicie"""
    arduino = serial.Serial(puerto, baudios)
    time.sleep(2)
    return arduino


def leer_valor_crudo(arduino):
    """Lee una línea del puerto, decodifica y convierte a entero"""
    try:
        linea = arduino.readline().decode('utf-8').strip()
        return int(linea)
    except:
        return None


def calcular_humedad(valor, wet, dry):
    """Convierte valor crudo en % de humedad"""
    porcentaje = (valor - wet) * 100 / (dry - wet)
    porcentaje = 100 - porcentaje  # invierte escala
    return max(0, min(100, round(porcentaje)))

In [ ]:
#Clases para gráfico y guardado
class GuardadorCSV:
    def __init__(self, archivo='humedad_datos.csv'):
        self.datos = []
        self.archivo = archivo

    def agregar(self, tiempo, humedad):
        self.datos.append({'tiempo': tiempo, 'humedad': humedad})

    def guardar(self):
        df = pd.DataFrame(self.datos)
        df.to_csv(self.archivo, index=False)
        print(f"✅ Datos guardados en {self.archivo}")

class GraficadorNotebook:
    def __init__(self, max_puntos=60):
        self.max_puntos = max_puntos
        self.tiempos = []
        self.humedades = []

    def actualizar(self, tiempo, humedad):
        # Añade y recorta a max_puntos
        self.tiempos.append(tiempo)
        self.humedades.append(humedad)
        if len(self.tiempos) > self.max_puntos:
            self.tiempos.pop(0)
            self.humedades.pop(0)

        # Limpia la salida anterior y dibuja
        clear_output(wait=True)
        plt.figure(figsize=(8,4))
        plt.plot(self.tiempos, self.humedades, marker='o')
        plt.title("Humedad en tiempo real")
        plt.xlabel("Hora")
        plt.ylabel("Humedad (%)")
        plt.ylim(0, 100)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
# Parámetros de calibración
WET = promedio_wet
DRY = 450
# Inicializar componentes
arduino = iniciar_arduino('COM7', 9600)
graf = GraficadorNotebook(max_puntos=60)
guarda = GuardadorCSV()
print("⏳ Leyendo datos (Ctrl+C para detener):")

try:
    while True:
        valor = leer_valor_crudo(arduino)
        if valor is not None:
            humedad = calcular_humedad(valor, WET, DRY)
            hora = datetime.now().strftime("%H:%M:%S")
            print(f"{hora} - Valor: {valor} → Humedad: {humedad}%")
            graf.actualizar(hora, humedad)
            guarda.agregar(hora, humedad)
        time.sleep(1)

except KeyboardInterrupt:
    guarda.guardar()
    arduino.close()
    print("🔚 Proceso finalizado.")